## Version check

Pytorch 를 연습하기 위하여, y = f(x) 형식의 regression 모델을 학습하는 뉴럴 네트워크를 만들어봅니다.

현재 실습의 torch 버전은 1.0.1 입니다.

In [1]:
import torch
print('PyTorch version = {}'.format(torch.__version__))

PyTorch version = 1.0.1.post2


Pytorch 에서 신경써야 할 요소는 네 가지 입니다.
  - data
  - model
  - loss function
  - optimizer

## Prepare data

아래는 거의 default 로 이용하는 요소들입니다.

In [2]:
# import torch components
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init

우리는 이번 실습에서 인공데이터를 만들어 이용할 것입니다.

TensorFlow 가 Tensor Board 를 제공하듯이 PyTorch 도 Visdom 을 제공합니다만, model 을 만드는 것부터 이야기합니다. 이번 실습에서는 scatter plot 만 그릴 것이기 때문에, 이전 시간에 살펴본 Bokeh 를 이용한 scatter plot 함수를 visualize.py 파일에 만들어 뒀습니다.

In [3]:
# import numpy for data generation
import numpy as np

# import visualizing functions
from bokeh.plotting import show
from visualize import scatter, mscatter

Loading BokehJS ...

Available output Bokeh figure in notebook


1000 개의 인공데이터를 만듭니다. x = [-10, 10] 사이에서 y = -x^2 를 따르도록 만들며, random noise 를 추가하였습니다.

In [4]:
# data generation
num_data = 1000 

noise = init.normal_(torch.FloatTensor(num_data,1), std=5)
x = init.uniform_(torch.Tensor(num_data,1), a=-10, b=10)
y = -x** 2
y = y + noise

x 는 torch.Tensor 입니다. numpy.ndarray.shape 과 같은 함수는 torch.Tensor.size() 입니다.

1 차원 데이터 1000 개가 만들어 졌습니다.

In [5]:
print(type(x))
print(x.size())

<class 'torch.Tensor'>
torch.Size([1000, 1])


Bokeh 를 이용하여 데이터의 scatter plot 을 그리려면 데이터가 numpy.ndarray 형식이어야 합니다.

Pytorch 의 Tensor 는 numpy 와 호환이 잘됩니다. .numpy() 함수를 이용하면 numpy.ndarray 가 됩니다.

x, y 모두 numpy() 를 이용하여 numpy.ndarray 로 만든 뒤, reshape(-1) 을 하여 column vector 로 만들어 줍니다.

In [6]:
p = scatter(x.numpy().reshape(-1), y.numpy().reshape(-1))
show(p)

## Linear regression

우리는 앞서 네 가지 요소 중 한가지인 데이터의 타입과 numpy 와의 호환에 대하여 살펴보았습니다. 

  - (v) data
  - ( ) model
  - ( ) loss function
  - ( ) optimizer

이제 나머지 요소들에 대하여 알아봅니다.

### model

y = a * x 식을 만들어 봅니다. 물론 위 데이터의 형태는 이차함수이기 때문에 일차함수인 y = a * x 가 학습될리 없습니다. 일단 만들어 봅시다.

torch.nn 에는 다양한 neural network 의 layer 들이 구현되어 있습니다. 이를 가져와 이용합니다. a 라는 parameter 가 저장되어 있는 linear layer 를 만듭니다. nn.Linear(a, b) 는 a 차원의 데이터가 b 차원으로 변환되는 (a, b) 크기의 행렬이라는 의미입니다.

### loss function

nn.MSELoss 는 Mean Squared Error loss 입니다. 실제 y 값과 예측된 y' 값의 $\sqrt{(y - y')}$ 입니다.

### optimizer

우리는 stochastic gradient descent optimizer 를 이용합니다. optimizer 를 만들 때에는 (1) 학습할 패러매터들과 (2) learning rate 를 설정해줘야 합니다.

    (1) model.parameters()
    (2) lr = YOUR VALUE

In [7]:
# model construction
model = nn.Linear(1,1)

# loss function & optimizer
# Mean Squared Error loss
loss_func = nn.MSELoss()

# Stochastic Gradient Descent optimizer
optimizer = optim.SGD(model.parameters(),lr=0.01)

## Train function

모든 요소들이 갖춰졌습니다. 한 번에 모든 데이터를 이용하여 모델을 업데이트 할 수도 있고, 부분씩 나눠서 모델을 업데이트 할 수도 있습니다. 

Mini batch 로 데이터를 나눠 학습하고 싶을 때 이를 직접 나눠서 구현할 수도 있으며, torch.data.DataLoader 를 이용할 수도 있습니다. 우리는 batch 로 학습하는 train 함수를 만듭니다.

아래의 과정이 standard 라고 생각하셔도 됩니다.

In [8]:
# define train function

def train(x, y, model, loss_func, optimizer, num_epoch):

    # output as Variable
    label = y
    
    # for given epochs
    for i in range(1, num_epoch + 1):

        # prediction
        output = model(x)

        # clears the gradients of all optimized
        optimizer.zero_grad()

        # compute loss
        loss = loss_func(output, label)

        # back-propagation
        loss.backward()

        # update model parameter
        optimizer.step()

        if i % 100 == 0:
            print('\riter = {}, loss = {}'.format(i, loss.data.numpy()), end='')
        if i % 1000 == 0:
            print()

    return model, output

학습 함수를 만들었으니, 네 가지 요소를 모두 입력하여 함수를 학습합니다. 우리는 총 1000 번 반복하여 모델의 패러메터, a 를 학습합니다.

In [9]:
model, output = train(x, y, model, loss_func, optimizer, num_epoch=1000)

iter = 1000, loss = 949.2531127929688


return 되는 output 은 모델에 의하여 prediction 이 이뤄진 y 값입니다. output 의 type 은 Tensor 입니다.

In [10]:
type(output)

torch.Tensor

그 안에는 data 가 있습니다. 그 역시 type 은 Tensor 이며, size() 를 확인해보면, 1 차원의 1000 개의 points 입니다.

In [11]:
print(type(output.data))
print(output.data.size())

<class 'torch.Tensor'>
torch.Size([1000, 1])


이를 numpy 로 변환하여 다시 한 번 시각화를 합니다.

In [12]:
# convert torch.Tensor to numpy.ndarray
output_numpy = output.data.numpy()
type(output_numpy)

numpy.ndarray

output_numpy 는 numpy.ndarray 이기 때문에 shape 을 이용할 수 있습니다.

In [13]:
output_numpy.shape

(1000, 1)

데이터의 분포에 맞지 않는 모델을 학습하였기 때문에 prediction 값이 엉망입니다.

In [14]:
y_pred = output.data

p = scatter(
    x.numpy().reshape(-1),
    y_pred.numpy().reshape(-1)
)

mscatter(
    p,
    x.numpy().reshape(-1),
    y.numpy().reshape(-1),
    fill_color='red'
)

show(p)

## Non-linear regression. Feed-forward neural network

이번에는 두 개의 hidden layer 를 쌓아서 이차함수의 분포를 학습해 봅니다.

여러 개의 layer 를 쌓을 때에는 nn.Sequential 을 이용하면 좋습니다. 순차적으로 모든 layer 가 적용된다는 의미입니다.

처음에 1 차원의 데이터를 20 차원으로 보내고, 20 차원의 데이터를 다시 5 차원으로 보낸 뒤, 이를 이용하여 y 를 prediction 하는 1 - 20 - 5 - 1 구조의 feed forward neural network 를 만듭니다. Activation function 으로 ReLU 를 이용합니다.

In [15]:
# model construction
model = nn.Sequential(
    
    # 1st hidden layer
    nn.Linear(1,20),
    
    # 1st activation function
    nn.ReLU(),
    
    # 2nd hidden layer
    nn.Linear(20,5),
    
    # 2st activation function
    nn.ReLU(),
    
    # last hidden layer
    # output is 1-dim for prediction
    nn.Linear(5,1),

)

optimizer 는 반드시 다시 만들어줘야 합니다. 각 optimizer 를 만들 때 우리는 학습할 패러매터 model.parameters() 를 argument 로 받았습니다. 새롭게 모델을 만들면, 그 모델의 parameters() 를 optimizer 에 연결해야 합니다.

In [16]:
# loss function & optimizer
# L1 loss
loss_func = nn.L1Loss()

# Adam optimizer
optimizer = optim.Adam(model.parameters(),lr=0.001)

이번에는 10K 번 학습을 합니다. 1000 번의 epoch 단위로 loss 값을 출력하도록 하였습니다. 조금씩 정확히 y 를 맞춰갑니다.

In [17]:
model, output = train(x, y, model, loss_func, optimizer, num_epoch=10000)

iter = 1000, loss = 7.198390483856201
iter = 2000, loss = 4.1908292770385745
iter = 3000, loss = 3.9579622745513916
iter = 4000, loss = 3.9368557929992676
iter = 5000, loss = 3.9320967197418213
iter = 6000, loss = 3.9279549121856697
iter = 7000, loss = 3.9249975681304936
iter = 8000, loss = 3.9244503974914554
iter = 9000, loss = 3.9237837791442875
iter = 10000, loss = 3.923490047454834


앞선 코드를 그대로 다시 이용하여 prediction 결과를 살펴봅니다. 이번에는 2 차 함수의 곡선을 거의 맞춰냈습니다.

In [18]:
y_pred = output.data

p = scatter(
    x.numpy().reshape(-1),
    y_pred.numpy().reshape(-1)
)

mscatter(
    p,
    x.numpy().reshape(-1),
    y.numpy().reshape(-1),
    fill_color='red'
)

show(p)

## Using GPU

Pytorch 에서 GPU 를 이용하려면 데이터와 모델에 cuda() 함수를 걸어주면 됩니다.

그 전에 해당 머신이 GPU 를 이용할 수 있는지 확인해야 합니다. 안전장치로 아래처럼 torch.cuda.is_available() 함수를 이용하여, GPU 를 사용할 것이고, 사용할 수 있을 때에만 조건적으로 cuda() 함수가 작동하도록 몇 줄만 추가하면 앞서 만든 train 함수를 GPU 용으로 만들 수 있습니다.

In [19]:
def train_w_gpu(x, y, model, loss_func, optimizer, num_epoch, use_gpu=True):

    if use_gpu and torch.cuda.is_available():
        model = model.cuda()
    
    # output as Variable
    if use_gpu and torch.cuda.is_available():
        label = y.cuda()
    else:
        label = y
    
    # for given epochs
    for i in range(1, num_epoch + 1):

        # prediction
        if use_gpu and torch.cuda.is_available():
            output = model(x.cuda())
        else:
            output = model(x)

        # clears the gradients of all optimized
        optimizer.zero_grad()

        # compute loss
        loss = loss_func(output,label)

        # back-propagation
        loss.backward()

        # update model parameter
        optimizer.step()

        if i % 100 == 0:
            if use_gpu and torch.cuda.is_available():
                loss_value = loss.data.cpu().numpy()
            else:
                loss_value = loss.data.numpy()
            print('\riter = {}, loss = {}'.format(i, loss_value), end='', flush=True)
        if i % 1000 == 0:
            print()

    if use_gpu and torch.cuda.is_available():
        output = output.cpu()

    return model, output

model, output = train_w_gpu(x, y, model, loss_func, optimizer, num_epoch=10000)

iter = 1000, loss = 3.923213243484497
iter = 2000, loss = 3.9229459762573245
iter = 3000, loss = 3.9227581024169923
iter = 4000, loss = 3.9226343631744385
iter = 5000, loss = 3.9231815338134766
iter = 6000, loss = 3.9222884178161626
iter = 7000, loss = 3.9221689701080322
iter = 8000, loss = 3.9227156639099127
iter = 9000, loss = 3.9228537082672124
iter = 10000, loss = 3.921921968460083


## Model 을 class 로 만들기

이전에는 model 을 nn.Sequential 의 형태로 만들었습니다. 같은 구조의 모델을 재활용하기 위해서는 해당 모델을 class 형태로 만들면 좋습니다.

Pytorch 에서 neural network 를 class 로 만들 때에는 두 가지 형식으로 만들 수 있습니다. 둘 모두 공통적으로 nn.Module 을 상속해야 합니다. 그리고 forward() 함수를 오버라이딩해야 합니다. nn.Module 을 상속하면, 이후 class instance 를 model 로 이용할 수 있습니다.

### type 1

일단 Python 의 class 상속이기 때문에 super().__init__() 을 반드시 구현해야 합니다.

첫번째 타입으로는, 앞서 만든 nn.Sequential() 을 class attribute 로 이용하는 것입니다. self.layers 에 앞서 만든 nn.Sequential 을 그대로 구현합니다.

forward 함수의 input argument 는 데이터 X 입니다. 이를 받아 self.layers 에 입력하여 forward 결과 값을 얻습니다. 이를 output 으로 return 합니다.

In [20]:
class FFN1_type1(nn.Module):
    # torch.nn 의 layers 로 이뤄진 모델로 만들기 위해서 
    # nn.Module 을 상속받아야 합니다.

    def __init__(self):        
        # 상속받은 class 를 생성하기 위해서는 아래처럼 super().__init__() 을 실행합니다.
        super(FFN1_type1, self).__init__()

        # model construction
        self.layers =  nn.Sequential(
            # 1st hidden layer
            nn.Linear(1,20),
            # 1st activation function
            nn.ReLU(),
            # 2nd hidden layer
            nn.Linear(20,5),
            # 2st activation function
            nn.ReLU(),
            # last hidden layer
            # output is 1-dim for prediction
            nn.Linear(5,1)
        )

    # feed-forward 를 위해서 forward() 함수를 overriding 합니다.
    def forward(self, x):
        output = self.layers(x)
        return output

앞서 만든 타입의 class 의 instance 를 만듭니다. 이것만으로 모델을 만드는 것이 모두 끝납니다. 그 뒤는 앞과 동일하게 train 함수를 이용합니다.

In [21]:
# model construction
model = FFN1_type1()

# loss function & optimizer
# L1 loss
loss_func = nn.L1Loss()
# Adam optimizer
optimizer = optim.Adam(model.parameters(),lr=0.001)

# train model
model, output = train(x, y, model, loss_func, optimizer, num_epoch=10000)

iter = 1000, loss = 33.730430603027344
iter = 2000, loss = 32.980560302734375
iter = 3000, loss = 32.296669006347656
iter = 4000, loss = 31.681135177612305
iter = 5000, loss = 31.110933303833008
iter = 6000, loss = 30.591102600097656
iter = 7000, loss = 30.103364944458008
iter = 8000, loss = 29.653665542602545
iter = 9000, loss = 29.230665206909187
iter = 10000, loss = 28.849462509155273


학습 결과도 제대로 되었음을 확인할 수 있습니다.

In [22]:
y_pred = output.data

p = scatter(
    x.numpy().reshape(-1),
    y_pred.numpy().reshape(-1)
)

mscatter(
    p,
    x.numpy().reshape(-1),
    y.numpy().reshape(-1),
    fill_color='red'
)

show(p)

### type 2

두번째 타입은 nn.Sequential 에 들어가던 각 요소를 따로따로 구현하는 것입니다. 이 때에는 forward 함수가 조금 복잡해집니다. 정확히는 nn.Sequential 에 들어가던 요소들을 풀어서 적용하는 것입니다. 

이번에는 activation function 이 layer 가 아니라 함수 형태로 이용됩니다. 그렇기 때문에 torch.nn.functional 을 import 합니다.

In [23]:
import torch.nn.functional as F

class FFN1_type2(nn.Module):
    # torch.nn 의 layers 로 이뤄진 모델로 만들기 위해서 
    # nn.Module 을 상속받아야 합니다.
    
    def __init__(self):        
        # 상속받은 class 를 생성하기 위해서는 아래처럼 super().__init__() 을 실행합니다.
        super(FFN1_type2, self).__init__()
        
        # 1st hidden layer
        self.fc1 = nn.Linear(1,20)
        
        # 2nd hidden layer
        self.fc2 = nn.Linear(20,5)

        # last hidden layer
        # output is 1-dim for prediction
        self.fc_out = nn.Linear(5,1)
    
    # feed-forward 를 위해서 forward() 함수를 overriding 합니다.
    # activation function 은 torch.nn 이 아닌
    # torch.nn.functional 을 이용합니다.
    def forward(self, x):

        # 1st hidden layer
        out = self.fc1(x)

        # 1st activation function
        out = F.relu(out)

        # 2nd hidden layer and activation
        out = F.relu(self.fc2(out))

        # to ouput layer
        out = self.fc_out(out)

        return out

두번째 타입의 class instance 를 만들어 학습을 합니다.

In [24]:
# model construction
model2 = FFN1_type2()

# loss function & optimizer
# L1 loss
loss_func = nn.L1Loss()
# Adam optimizer
optimizer = optim.Adam(model2.parameters(),lr=0.001)

# train model
model2, output = train(x, y, model2, loss_func, optimizer, num_epoch=10000)

iter = 1000, loss = 8.240599632263184
iter = 2000, loss = 4.4990673065185555
iter = 3000, loss = 3.9837970733642584
iter = 4000, loss = 3.9413194656372072
iter = 5000, loss = 3.9314763545989993
iter = 6000, loss = 3.9282135963439945
iter = 7000, loss = 3.9260275363922128
iter = 8000, loss = 3.9244666099548343
iter = 9000, loss = 3.9237902164459237
iter = 10000, loss = 3.9234724044799805


그리고 결과도 확인합니다. 둘 모두 학습이 잘 되었습니다.

In [25]:
y_pred = output.data

p = scatter(
    x.numpy().reshape(-1),
    y_pred.numpy().reshape(-1)
)

mscatter(
    p,
    x.numpy().reshape(-1),
    y.numpy().reshape(-1),
    fill_color='red'
)

show(p)

## model print

두 종류로 모델을 구현하였을 때 성능에는 차이가 없습니다. 단, 첫번째 타입에서는 activation function 도 neural network layer 취급이 됩니다.

nn.Module 을 print 하면 class 내부의 torch.nn 들이 출력됩니다.

In [26]:
print(model)

FFN1_type1(
  (layers): Sequential(
    (0): Linear(in_features=1, out_features=20, bias=True)
    (1): ReLU()
    (2): Linear(in_features=20, out_features=5, bias=True)
    (3): ReLU()
    (4): Linear(in_features=5, out_features=1, bias=True)
  )
)


In [27]:
print(model2)

FFN1_type2(
  (fc1): Linear(in_features=1, out_features=20, bias=True)
  (fc2): Linear(in_features=20, out_features=5, bias=True)
  (fc_out): Linear(in_features=5, out_features=1, bias=True)
)


## Get parameters

optimizer 에 입력하던 model.parameters() 는 실제 layer 에 저장된 값입니다. 우리가 학습된 모델을 다른 곳에 이식한다면, 이 값이 필요합니다.

그런데 이 때에는 첫번째, 두번째 type 모두 activation function 도 parameters 로 출력됩니다. 쌍으로 (layer, activation) 입니다.

In [28]:
parameters = list(model2.parameters())

2 개의 hidden layer, 1 개의 output layer 가 있었기 때문에 총 6 개의 layers 가 parameters 로 만들어집니다.

In [29]:
len(parameters)

6

각각의 layer 의 size 를 확인할 수 있습니다.

In [30]:
for i, parameter in enumerate(parameters):
    print('\n{} th layer'.format(i))
    print(type(parameter))
    print(parameter.size())


0 th layer
<class 'torch.nn.parameter.Parameter'>
torch.Size([20, 1])

1 th layer
<class 'torch.nn.parameter.Parameter'>
torch.Size([20])

2 th layer
<class 'torch.nn.parameter.Parameter'>
torch.Size([5, 20])

3 th layer
<class 'torch.nn.parameter.Parameter'>
torch.Size([5])

4 th layer
<class 'torch.nn.parameter.Parameter'>
torch.Size([1, 5])

5 th layer
<class 'torch.nn.parameter.Parameter'>
torch.Size([1])


첫 번째 hidden layer 의 학습된 parameter 값입니다.

In [31]:
parameters[0].data.numpy()

array([[ 1.0069038 ],
       [-0.6488291 ],
       [-0.4380213 ],
       [-0.37485817],
       [ 1.1819209 ],
       [ 1.1533408 ],
       [-0.7821023 ],
       [ 0.64279705],
       [-0.7592641 ],
       [-0.61557776],
       [ 0.3255952 ],
       [ 0.38282624],
       [ 0.40262616],
       [-0.35437056],
       [-0.9587932 ],
       [ 0.97473705],
       [-0.29412025],
       [-1.1960163 ],
       [-1.1228583 ],
       [-1.0446827 ]], dtype=float32)

두 번째 hidden layer 의 학습된 parameter 값입니다. 

In [32]:
parameters[2].data.numpy()

array([[ 0.08238118, -0.09378945, -0.21451661,  0.0714435 ,  0.02653917,
         0.03684681, -0.5732388 ,  0.86364645, -0.667664  , -0.41329437,
         0.95490116,  1.4865568 ,  0.6791376 , -0.04992038, -0.57627535,
         0.7528278 , -0.40299642, -0.391516  , -0.4464575 , -0.31906828],
       [ 0.22835948,  0.7510188 ,  0.7969089 ,  0.5579024 ,  0.5372791 ,
         0.46390927,  0.47158128,  0.6133131 , -0.05740607,  0.62654245,
         0.90926474,  1.447403  ,  0.5799547 ,  1.8063073 ,  0.23899978,
         0.82361466,  0.8469997 ,  0.2639937 ,  0.43468878,  0.42705274],
       [ 0.23600258, -0.53622675, -0.5032731 ,  0.04095028,  0.39693716,
        -0.00671157, -0.00488812, -0.17832151,  0.03863805, -0.6074972 ,
        -1.5534173 , -2.3230612 , -0.85318726, -2.1534767 ,  0.0720914 ,
         0.00537452, -1.1395208 ,  0.33150098,  0.37445506,  0.44952103],
       [ 0.21627013,  0.80991375,  0.9080119 ,  0.60840756,  0.37264973,
         0.4103319 ,  0.2994296 ,  0.9533467 ,  